In [1]:
%pip install --only-binary=:all: tiktoken
%pip install --upgrade pip setuptools wheel
%pip install tiktoken --only-binary=:all:

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Install required libraries
%pip install -qU \
    langchain==0.3.* \
    langchain_openai==0.3.* \
    langchain_community \
    unstructured[md]==0.17.* \
    langgraph==0.4.* \
    websockets==15.0.*

Note: you may need to restart the kernel to use updated packages.


In [10]:
import langchain

from langchain_community.utilities.requests import RequestsWrapper
from langchain_community.agent_toolkits.openapi import planner
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec
import requests

from langchain_community.document_loaders.text import TextLoader
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableMap, RunnableAssign
from langchain_core.documents.base import Document

from dotenv import load_dotenv
from pprint import pp
from IPython.display import Markdown as render_md
import pynvml  # type: ignore[import]
from dotenv import load_dotenv

import os

# openai api need to be set up before runnung the below cell. 

import getpass # for testing. remove for compilation 

os.environ['OPENAI_API_KEY'] = getpass.getpass("Enter your OpenAI API key: ")


def get_supporting_documents():
    current = os.getcwd()

    while True:
        candidate = os.path.join(current, 'GAI-3101-CAP')
        if os.path.isdir(candidate):
            target = os.path.join(candidate, 'capstone/support-info')
            return os.path.relpath(target, os.getcwd())

        parent = os.path.dirname(current)
        if parent == current:
            break  # Reached root without finding 'GAI-3101-CAP'
        current = parent

    raise FileNotFoundError("Could not find a directory containing 'GAI-3101-CAP'")

SUPPORT_DIR = get_supporting_documents()


def get_files_with_extensions(folder_path, extensions):
    matched_files = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if any(file.lower().endswith(ext.lower()) for ext in extensions):
                matched_files.append(os.path.join(root, file))
    return matched_files

get_files_with_extensions(SUPPORT_DIR, ['.md'])

md_paths = get_files_with_extensions(SUPPORT_DIR,['.md'])
md_docs = []
for path in md_paths:
    text_loader = TextLoader(path)
    doc = text_loader.load()
    md_docs += doc


embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small",
)
vector_store = InMemoryVectorStore(embedding_model)
vector_store.add_documents(documents=md_docs)


doc_retriever = vector_store.as_retriever(
    search_type="similarity", #could also do MMR if we want to avoid repeated/very similar docs
    search_kwargs={
        'k': 3 # number of documents to consider
    }
)

system_msg = '''
You are a helpful AI bot being used to assist Beanbotics., a company that sell automated robotic arms that make coffee.
Your job is to automate the initial processing of
support tickets. You will read incoming tickets, extract key information,
classify the requests into categories, assign priority levels, and update the ticketing
system with this information

Instructions:
- Use only the content found in the provided context documents unless explicitly asked to use outside information
- Include inline citations using the `source` field from metadata (e.g., [source]).
- If documents conflict, describe the differing perspectives and cite each source (e.g., [source A], [source B]).
- If the answer cannot be found, say: “The context does not contain that information.”
- If the question is ambiguous, explain your interpretation before answering.'
'''

human_msg = """
Please answer this query from the user
<original user query>
{query}
</original user query>

---
Use these docs to answer the original user query
<context documents>
{context_docs}
</context documents>
"""


def render_and_pass(x):
    display(render_md(x))
    
render_output = RunnableLambda(render_and_pass)

llm = ChatOpenAI(
    model="gpt-4o-mini",
    # max_tokens=200,
    max_retries=2, 
)


generation_template = ChatPromptTemplate([
    ("system", system_msg),
    ("human", human_msg),
])

rag_chain = (
    RunnableMap(
        query=RunnablePassthrough(),
        context_docs=doc_retriever
    )
    | RunnableMap(
        context_docs=RunnableLambda(lambda x: x['context_docs']),
        llm_response=(
            generation_template
            | llm
            | StrOutputParser()
            | render_output
        )
    )
)

rag_result = rag_chain.invoke("What does this product do?")

print(rag_result["llm_response"])

#enriched_instructions = f"""
#Use the following documentation context to complete the task:

#{rag_result['llm_response']}

#Original instructions:
#{instructions}
#"""

The context documents do not provide specific information about what the product does directly. They primarily contain internal support guidelines related to coffee quality, cleaning, maintenance, and feature requests for the Bean Machine, which is a coffee-making product by Beanbotics. 

To summarize from the available documents:
- The Bean Machine is used for preparing coffee, and support documents outline how to troubleshoot issues related to coffee quality, maintenance, and features. However, there is no clear description of its functions or features in the context provided.

Thus, the context does not contain a direct answer to the question regarding what the product does.

None


In [ ]:
from langchain_community.utilities.requests import RequestsWrapper
from langchain_community.agent_toolkits.openapi import planner
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec
import requests
import os
import json
import websockets
import requests
import asyncio

from dotenv import load_dotenv

# Load variables from .env file
load_dotenv()

openai_api_key = os.environ["OPENAI_API_KEY"]
if (openai_api_key):
    print("------------------- Loaded Key -------------------")
# print(openai_api_key)  # optional: check it loaded correctly

# Load the OpenAPI specification from the running ticketing system
root = "http://localhost:3000"
api_spec_url = f"{root}/api/docs/openapi.json"

# Download and parse the OpenAPI spec
response = requests.get(api_spec_url)
data = response.json()
data['servers'] = [{'url': root}]
openapi_spec = reduce_openapi_spec(data, dereference=False)

### Try to get it to categorize all tickets
requests_wrapper = RequestsWrapper()
llm = ChatOpenAI(model_name="gpt-4o", temperature=0.0)

agent = planner.create_openapi_agent(
    api_spec=openapi_spec,
    requests_wrapper=requests_wrapper,
    llm=llm,
    verbose=True,
    allow_dangerous_requests=True,
    handle_parsing_errors=True,
    allow_operations=['GET', 'POST', 'PUT', 'PATCH', 'DELETE']
)

response = requests.get(root+ "/api/tickets")
tickets = response.json()

WS_URL = "ws://localhost:3000/ws"
# This async function connects to the WebSocket and listens for ticket updates
# Once a ticket update is received, it yields it for processing.
async def listen_for_ticket_updates():
    print("\n------------------- Starting Connection and Agent -------------------")
    # Establish a connection to the WebSocket server
    async with websockets.connect(WS_URL) as websocket:
        print("WebSocket connection established.")
        try:
            # Keep listening for messages from the server
            while True:
                message = await websocket.recv()  # Wait for a new message
                yield json.loads(message)
        except websockets.ConnectionClosed:
            print("WebSocket connection closed.")
        except Exception as e:
            print(f"WebSocket error: {e}")

# --- Async wrapper for the agent ---
async def run_agent_async(instructions: str):
    """Run the agent in a non-blocking way."""
    # agent.invoke is synchronous, so run it in a thread
    return await asyncio.to_thread(agent.invoke, instructions)

# --- Async agent helper ---
async def run_agent(ticket_id, action_description, instructions):
    """Run an agent action asynchronously with logging."""
    print(f'{action_description} for ticket: {ticket_id}')
    return await run_agent_async(instructions.strip())


# --- Simplified async ticket functions ---
async def determine_category(ticket_id):
    await run_agent(
        ticket_id,
        "Categorizing ticket",
        f"""
Based on context from this ticket {ticket_id} give it one category from: General Inquiry, Mechanical, Quality, Maintenance, Technical, Awaiting Details.
Use “Awaiting Details” ONLY if there is insufficient info or very vague . Then POST the assigned category:
POST /api/tickets/{ticket_id}/category with the category in the body

"""
    )

async def determine_priority(ticket_id):
    await run_agent(
        ticket_id,
        "Determining ticket priority",
        f"""
Assign this ticket {ticket_id} a priority: High, Medium, or Low based ond details from the ticket. 
If the category is “Awaiting Details,” set priority to Low. Otherwise, POST the assigned priority:
POST /api/tickets/{ticket_id}/priority with the priority set to "Low"

"""
    )

async def determine_response(ticket_id):
    await run_agent(
        ticket_id,
        "Checking for automatic response",
        f"""
Check out the ticket ticket {ticket_id}. 
Post a detailed response based on this troubleshooting guide {rag_result}
Use JSON: ticket_id={ticket_id}, author="Support Agent", message="<friendly message>"
"""
    )

async def determine_status(ticket_id):
    await run_agent(
        ticket_id,
        "Checking ticket status",
        f"""
Check ticket {ticket_id}. 
If priority is High and the information in the ticket seems urgent, 
AND if there is no response from an Agent already then
update status to "escalated".
If there is a response from an agent, change the status to in progress
Otherwise, do nothing.
"""
    )

async def determine_escalation(ticket_id):
    await run_agent(
        ticket_id,
        "Checking if escalation is required",
        f"""
Check ticket {ticket_id}. 
If status is escalated, POST an automatic escalation response: 
ticket_id={ticket_id}, author="Support Agent", message="<context-aware escalation message>"
"""
    )

async def escalate_from_response(ticket_id):
    await run_agent(
        ticket_id,
        "Check if escalation is needed",
        f"""Analyze the latest response for ticket {ticket_id}. 
        If urgent and cannot be auto-handled, update ticket status to 'escalated'. Otherwise, do nothing.
        POST /api/tickets/{ticket_id}/status"""
    )

async def auto_respond(ticket_id):
    await run_agent(
        ticket_id,
        "Auto-response check",
        f"""If ticket {ticket_id} is escalated, send response based on context and acknowledge escalated status. Otherwise, send a friendly response as Support Agent: 
        Follow this format ticket_id={ticket_id}, author='Support Agent', message='<friendly placeholder>'."""
    )

recent_ticket = "" # Cheap way to shortcut it, doesn't work 100% of the time

# --- Async listener loop ---
async for message in listen_for_ticket_updates():
    ticket_id = message.get('ticketId')
    update_type = message.get('updateType')
    print("----------------- MESSAGE -------------")
    print(message)
    print("----------------- Ticket ID -------------")
    print(ticket_id)
    print("----------------- UPDATE TYPE -------------")
    print(update_type)

    if update_type == 'created':
        print("CREATED")
        # Run all ticket handlers asynchronously in sequence
        await determine_category(ticket_id)
        await determine_priority(ticket_id)
        await determine_response(ticket_id)
        await determine_status(ticket_id)
        await determine_escalation(ticket_id)

    elif update_type == 'response' and ticket_id != recent_ticket:
        await escalate_from_response(ticket_id)
        await auto_respond(ticket_id)
        

    recent_ticket = ticket_id

------------------- Loaded Key -------------------

------------------- Starting Connection and Agent -------------------
WebSocket connection established.
----------------- MESSAGE -------------
{'ticketId': '02d4f6ed-0dec-4576-997f-c43c0de2ab4c', 'updateType': 'response'}
----------------- Ticket ID -------------
02d4f6ed-0dec-4576-997f-c43c0de2ab4c
----------------- UPDATE TYPE -------------
response
Check if escalation is needed for ticket: 02d4f6ed-0dec-4576-997f-c43c0de2ab4c


> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API calls to analyze the latest response for ticket 02d4f6ed-0dec-4576-997f-c43c0de2ab4c and update its status to 'escalated' if urgent and cannot be auto-handled.
Observation: 1. **GET /api/tickets/02d4f6ed-0dec-4576-997f-c43c0de2ab4c**: Retrieve the details of the ticket with ID 02d4f6ed-0dec-4576-997f-c43c0de2ab4c to analyze the latest response and determine if the ticket is urgent and cannot be auto-handled.